<a href="https://colab.research.google.com/github/navroz-lamba/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/LS_DS_122_Chi2_Tests_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - Practice Chi^2 Tests

Use the following dataset relating to math scores of students in two different Portugese schools:

<https://archive.ics.uci.edu/ml/datasets/Student+Performance>

### 1) Load the dataset specifically relating to math scores as a new dataframe.

There are two datasets in the `student.zip` file, make sure you use `student-mat.csv`.


In [ ]:
# YOUR WORK HERE
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip

--2020-07-14 17:41:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20478 (20K) [application/x-httpd-php]
Saving to: ‘student.zip’

student.zip         100%[===================>]  20.00K  --.-KB/s    in 0.1s    

2020-07-14 17:41:54 (154 KB/s) - ‘student.zip’ saved [20478/20478]



In [17]:
!unzip student.zip

Archive:  student.zip
replace student-mat.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace student-por.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: [n]
error:  invalid response [[n]]
replace student-por.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [18]:
import pandas as pd
df = pd.read_csv('student-mat.csv', sep=';')

print(df.shape)
df.head()

(395, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


### 2) Use Chi^2 tests and `stats.chi2_contingency()` to identify:
 - Two pairs of variables that are dependent (are associated with one another).
 - Two pairs of variables that are independent (have no significant relationship).

Does it make intuitive sense why the variables in these pairs might or might not show a relationship to one another? 


In [62]:
import scipy.stats as stats

In [ ]:
df.describe(exclude='number')

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
count,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395
unique,2,2,2,2,2,5,5,4,3,2,2,2,2,2,2,2,2
top,GP,F,U,GT3,T,other,other,course,mother,no,yes,no,yes,yes,yes,yes,no
freq,349,208,307,281,354,141,217,145,273,344,242,214,201,314,375,329,263


In [71]:
# Run 1 - Independent 
observed = pd.crosstab(df['guardian'], df['internet'])
observed.values

array([[ 12,  78],
       [ 47, 226],
       [  7,  25]])

Null Hypothesis: Independent

Alternative Hypothesis: Depenedent 

Confidence Level = 95%

In [67]:
chi2, p_value, dof, expected = stats.chi2_contingency(observed)
print(f'chi2: {chi2}\n p_value: {p_value}\n dof:{dof}\n expected:{expected}')

chi2: 1.4009986604496485
 p_value: 0.49633740563669426
 dof:2
 expected:[[ 15.03797468  74.96202532]
 [ 45.61518987 227.38481013]
 [  5.34683544  26.65316456]]


CONCLUSION:

Considering the p_value of 0.4963, we fail to reject the Null Hypothesis.

In [70]:
# Run 2 - Dependent
observed1 = pd.crosstab(df['Mjob'], df['Fjob'])
observed1.values

array([[  7,   2,  33,  15,   2],
       [  0,   6,  17,  10,   1],
       [  5,   2, 104,  24,   6],
       [  6,   4,  42,  43,   8],
       [  2,   4,  21,  19,  12]])

Null Hypothesis: Independent

Alternative Hypothesis: Depenedent 

Confidence Level = 95%

In [72]:
chi2, p_value, dof, expected = stats.chi2_contingency(observed1)
print(f'chi2: {chi2}\n p_value: {p_value}\n dof:{dof}\n expected:{expected}')

chi2: 73.38089527305267
 p_value: 2.533576541234461e-09
 dof:16
 expected:[[ 2.98734177  2.68860759 32.41265823 16.57974684  4.33164557]
 [ 1.72151899  1.54936709 18.67848101  9.55443038  2.49620253]
 [ 7.13924051  6.42531646 77.46075949 39.62278481 10.35189873]
 [ 5.21518987  4.69367089 56.58481013 28.9443038   7.56202532]
 [ 2.93670886  2.64303797 31.86329114 16.29873418  4.25822785]]


CONCLUSION:

Considering the p_value ~0 we reject the Null Hypothesis and suggest alternate Hypothesis that the variables are dependent.

In [84]:
# Run 3 - Dependent
observed2 = pd.crosstab(df['paid'], df['failures'])
observed2.values

array([[153,  37,  10,  14],
       [159,  13,   7,   2]])

Null Hypothesis: Independent

Alternative Hypothesis: Depenedent 

Confidence Level = 95%

In [85]:
chi2, p_value, dof, expected = stats.chi2_contingency(observed2)
print(f'chi2: {chi2}\n p_value: {p_value}\n dof:{dof}\n expected:{expected}')

chi2: 18.537217658927684
 p_value: 0.0003407451938069661
 dof:3
 expected:[[169.03291139  27.08860759   9.21012658   8.66835443]
 [142.96708861  22.91139241   7.78987342   7.33164557]]


CONCLUSION:

Considering the p_value ~0 we reject the Null Hypothesis and suggest alternate Hypothesis that the variables are dependent.

In [88]:
# Run 4 - Independent 
observed3 = pd.crosstab(df['guardian'], df['paid'])
observed3.values

array([[ 54,  36],
       [142, 131],
       [ 18,  14]])

Null Hypothesis: Independent

Alternative Hypothesis: Depenedent 

Confidence Level = 95%

In [89]:
chi2, p_value, dof, expected = stats.chi2_contingency(observed3)
print(f'chi2: {chi2}\n p_value: {p_value}\n dof:{dof}\n expected:{expected}')

chi2: 1.798816528947036
 p_value: 0.40681031264667245
 dof:2
 expected:[[ 48.75949367  41.24050633]
 [147.90379747 125.09620253]
 [ 17.33670886  14.66329114]]


CONCLUSION:

Considering p_value we fail to reject the Null Hypothesis

###3) Use NumPy to perform your own chi^2 test "from scratch"
Pick any of the chi^2 tests that you ran in part 2 and try them on your own. You should get the same results that Scipy got for all four values returned from chi2_contingency()

In [143]:
# Run 1
observed = pd.crosstab(df['guardian'], df['internet'])
observed.shape

(3, 2)

In [164]:
contingency_table = pd.crosstab(df['guardian'], df['internet'], margins=True)
contingency_table

internet,no,yes,All
guardian,,,
father,12,78,90
mother,47,226,273
other,7,25,32
All,66,329,395


In [24]:
# Expected Value = (row_totals * column_totals) / total_observations
father_no_internet = (90 * 66) / 395
father_no_internet

15.037974683544304

In [151]:
# making a table of expected values 
# we would need row_totals
# Col_totals
# Total observations


row_totals = df['internet'].value_counts().sort_index().values
row_totals

array([ 66, 329])

In [152]:
col_totals = df['guardian'].value_counts().sort_index().values
col_totals

array([ 90, 273,  32])

In [42]:
# finding total observations
total_observations = df.shape[0]
total_observations

395

In [153]:
import numpy as np
expected = np.array([[0.0, 0.0],
                     [0.0, 0.0],
                     [0.0, 0.0]])

for i, col in enumerate(col_totals): 
  for j, row in enumerate(row_totals):
    expected_value = row*col / total_observations
    expected[i][j] = expected_value

expected 

array([[ 15.03797468,  74.96202532],
       [ 45.61518987, 227.38481013],
       [  5.34683544,  26.65316456]])

In [54]:
chi2 = ((observed - expected) ** 2 / expected ).sum().sum()
chi2

1.4009986604496487

In [56]:
# DOF = (# rows_crosstab - 1) * (# cols_crosstab - 1)
dof = (3 - 1) * (2 -1) # shape being (3,2)
dof

2

In [ ]:
p-value = 0.49634 # The p-value matches the one calculated with scipy.stats
# We fail to reject the hypothesis 

## Stretch goals:

### 1. Refactor your code so it is elegant, readable, and holds reusable code in functions.

In [138]:
def chi2_dof_value(df, attribute1, attribute2):

# Given information
  observed = pd.crosstab(df[attribute2], df[attribute1])             
# Sample Size 
  total_observations = df.shape[0]                
# Total number of Rows
  row_totals = df[attribute1].value_counts().sort_index().values
# Total number of Columns 
  col_totals = df[attribute2].value_counts().sort_index().values
# making expected values as an array with same shape as of observed filled with 0.0
  expected = np.array(observed) * 0.0
# running nested loops to iterate over every combination of cols and rows 
  for i, col in enumerate(col_totals): 
    for j, row in enumerate(row_totals):
      expected_value = row*col / total_observations
      expected[i][j] = expected_value
# using the simple formula to calculate chi2
  chi2 = ((observed - expected) ** 2 / expected ).sum().sum()
# using the formula to calculate degree of freedom
  dof = (observed.shape[0] - 1) * (observed.shape[1] -1)

  print(f'chi2: {chi2}\n\ndof: {dof}')



In [139]:
# Pass Three arguments : dataframe and the two attributes we are doing hypothesis on 
chi2_dof_value(df, 'internet', 'guardian') 

chi2: 1.4009986604496487

dof: 2




### 2. Check For Understanding - Study and write your own explanations/definitions for these topics:

- What is a sample "estimate" in statistics?

- What are hypothesis test? How are they useful?

- What is a "null hypothesis?"

- What is a p-value and what does it represent?

- What does it mean for something to be "statistically significant?"

- What is a test statistic and how does it relate to a p-value?

- What are "degrees of freedom" and how are they calculated in a 1-sample, 2-sample, and chi^2 test for independence?

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)